In [9]:
!pip install pydeck geopandas -q

In [2]:
!pip install --upgrade 'google-cloud-bigquery[bqstorage,pandas]' -q

In [1]:
%load_ext google.cloud.bigquery

In [2]:
import pydeck as pdk
import geopandas as gpd
from shapely import wkt
import matplotlib
import matplotlib.cm as cm

In [11]:
%%bigquery energy_labels_neighbourhood
with energy_neighbourhood as (

    select 
        dim_neighbourhood.neighbourhood_key,
        avg(fct_energy_label.energy_label_numeric) as mean_energy_label
    from 
        `quantile.analytics.fct_energy_label` fct_energy_label
    inner join
        `quantile.analytics.dim_neighbourhood` dim_neighbourhood
    on 
        fct_energy_label.neighbourhood_key = dim_neighbourhood.neighbourhood_key
    group by 
        dim_neighbourhood.neighbourhood_key

)

select 
    energy_neighbourhood.mean_energy_label,
    dim_neighbourhood.neighbourhood_name,
    dim_neighbourhood.municipality_name,
    ST_SIMPLIFY(dim_neighbourhood.neighbourhood_geometry, 10) as neighbourhood_geometry
from 
    energy_neighbourhood 
left join 
    `quantile.analytics.dim_neighbourhood` dim_neighbourhood
on 
    energy_neighbourhood.neighbourhood_key = dim_neighbourhood.neighbourhood_key

Downloading: 100%|██████████| 13459/13459 [00:02<00:00, 5247.41rows/s]


In [12]:
energy_labels_neighbourhood.head()

,mean_energy_label,neighbourhood_name,municipality_name,neighbourhood_geometry
0,6.041596,Noorderplassen,Almere,GEOMETRYCOLLECTION(LINESTRING(5.161633730905 5...
1,8.000000,Schoppenstee,Hof van Twente,"POLYGON((6.70810404503765 52.262033408428, 6.7..."
2,7.589744,Ten Esschen,Heerlen,"POLYGON((5.91636675938768 50.9099181216575, 5...."
3,6.486486,Uitdam,Waterland,GEOMETRYCOLLECTION(LINESTRING(5.07684573198345...
4,9.566887,Bloemendaal,Bloemendaal,"POLYGON((4.61023962190999 52.4118098952414, 4...."


In [13]:
energy_labels_neighbourhood.to_csv('data/energy_labels_neighbourhood.csv', index=False)